In [8]:
from __future__ import annotations

from typing import Dict, Tuple

import toolpy as tp
from toolpy.tool.tool import TextLike

In [9]:
import os
from toolpy.integrations import groq

In [10]:
os.environ["GROQ_API_KEY"] = "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"

In [11]:
groq_interface = groq.GroqInterface(model=groq.GroqModel.LLAMA3_70B, n_retry=5)

registry = tp.llm.LLMRegistry()
registry.registry(model_name="llama3-70b", interface=groq_interface, default=True)

In [12]:
class QuestionGenerator(tp.BasicTool):
    _description = "Generates questions from a list of memories"
    _input_description = {"memories": "a list of memory descriptions"}

    _system_message = '''You are a question generator that outputs in JSON.
The JSON must use the schema: {'questions': ['str', 'str', 'str']}. 

Please use a valid JSON format.'''

    _base_prompt = '''
Given the following list of memories:
{memories}

What are 3 most salient high-level questions we can answer about the subjects in these statements?'''

    _return_description = {'questions': 'a list of three high-level questions'}

    def __init__(self, model_name: str | None = None) -> None:
        super().__init__(description=self._description,
                         input_description=self._input_description,
                         prompt_template=self._base_prompt,
                         return_description=self._return_description,
                         system_message=self._system_message,
                         model_name=model_name,
                         json_mode=True)
        
    def _execute(self, query: Dict[str, str], context: str) -> Tuple[Dict[str, TextLike], Dict[str, str]]:
        result, return_description = super()._execute(query, context)
        
        questions = result["questions"]
        
        return {"questions": questions}, return_description


In [13]:
from __future__ import annotations

from collections import deque
from typing import List

import cst_python as cst

# from .question_generator_tool import QuestionGenerator

class QuestionGeneratorCodelet(cst.Codelet):
    def __init__(self, model_name: str | None = None, 
                 memories_name: str | None = None, 
                 generated_questions_name: str | None = None) -> None:
        super().__init__()

        if memories_name is None:
            memories_name = "Memories"
        if generated_questions_name is None:
            generated_questions_name = "GeneratedQuestions"

        self._memories_name = memories_name
        self._generated_questions_name = generated_questions_name

        self._question_generator = QuestionGenerator(model_name)

    def access_memory_objects(self) -> None:
        self._memories_mo = self.get_input(name=self._memories_name)
        self._generated_questions_mo = self.get_output(name=self._generated_questions_name)

    def calculate_activation(self) -> None:
        pass

    def proc(self) -> None:
        memories: List[str] = self._memories_mo.get_info()
        query = {"memories": "\n".join(memories)}
        result, _ = self._question_generator._execute(query, context="")
        questions = result["questions"]

        self._generated_questions_mo.set_info(questions)

In [14]:
from __future__ import annotations

from typing import Dict, Tuple, List

import toolpy as tp
from toolpy.tool.tool import TextLike

class InsightGenerator(tp.BasicTool):
    _description = "Generates high-level insights from a list of memory descriptions"
    _input_description = {"statements": "a list of statements/memory descriptions about the agent"}

    _system_message = '''You are an insight generator that outputs in JSON.
The JSON must use the schema: {'insights': ['str', 'str', 'str', 'str', 'str']}. 

Please use a valid JSON format.'''

    _base_prompt = '''
Statements about the agent:
{statements}

What 5 high-level insights can you infer from the above statements? (example format: insight (because of 1, 5, 3))'''

    _return_description = {'insights': 'a list of five high-level insights'}

    def __init__(self, model_name: str | None = None) -> None:
        super().__init__(description=self._description,
                         input_description=self._input_description,
                         prompt_template=self._base_prompt,
                         return_description=self._return_description,
                         system_message=self._system_message,
                         model_name=model_name,
                         json_mode=True)

    def _execute(self, query: Dict[str, str], context: str) -> Tuple[Dict[str, TextLike], Dict[str, str]]:
        result, return_description = super()._execute(query, context)
        
        insights = result["insights"]
        
        return {"insights": insights}, return_description

In [15]:
from __future__ import annotations

from collections import deque
from typing import List

import cst_python as cst

# from .insight_generator_tool import InsightGenerator

class InsightGeneratorCodelet(cst.Codelet):
    def __init__(self, model_name: str | None = None, 
                 statements_name: str | None = None, 
                 generated_insights_name: str | None = None) -> None:
        super().__init__()

        if statements_name is None:
            statements_name = "Statements"
        if generated_insights_name is None:
            generated_insights_name = "GeneratedInsights"

        self._statements_name = statements_name
        self._generated_insights_name = generated_insights_name

        self._insight_generator = InsightGenerator(model_name)

    def access_memory_objects(self) -> None:
        self._statements_mo = self.get_input(name=self._statements_name)
        self._generated_insights_mo = self.get_output(name=self._generated_insights_name)

    def calculate_activation(self) -> None:
        pass

    def proc(self) -> None:
        statements: List[str] = self._statements_mo.get_info()
        query = {"statements": "\n".join(statements)}
        result, _ = self._insight_generator._execute(query, context="")
        insights = result["insights"]

        self._generated_insights_mo.set_info(insights)

In [16]:
# Define the sample input for memories and statements
sample_memories = [
    "I graduated from college with honors.",
    "I traveled to Japan and experienced the cherry blossom festival.",
    "I started a new job in a field I'm passionate about."
]

sample_statements = [
    "Klaus Mueller is writing a research paper.",
    "Klaus Mueller enjoys reading a book on gentrification.",
    "Klaus Mueller is conversing with Ayesha Khan about exercising.",
    "Klaus Mueller recently attended a conference on urban development.",
    "Klaus Mueller often discusses social issues with friends."
]

# Initialize memory objects (assuming get_input and get_output are methods to interact with memory objects)
class MockMemoryObject:
    def __init__(self):
        self._info = []

    def get_info(self):
        return self._info

    def set_info(self, info):
        self._info = info

memories_mo = MockMemoryObject()
memories_mo.set_info(sample_memories)

generated_questions_mo = MockMemoryObject()

statements_mo = MockMemoryObject()
statements_mo.set_info(sample_statements)

generated_insights_mo = MockMemoryObject()

# Initialize the Codelet instances
question_generator_codelet = QuestionGeneratorCodelet(model_name="gpt-4")
insight_generator_codelet = InsightGeneratorCodelet(model_name="gpt-4")

# Set the memory objects for the QuestionGeneratorCodelet
question_generator_codelet._memories_mo = memories_mo
question_generator_codelet._generated_questions_mo = generated_questions_mo

# Execute the QuestionGeneratorCodelet
question_generator_codelet.proc()

# Retrieve the generated questions
generated_questions = generated_questions_mo.get_info()

print("Generated Questions:")
for question in generated_questions:
    print(f"- {question}")

# Set the memory objects for the InsightGeneratorCodelet
insight_generator_codelet._statements_mo = statements_mo
insight_generator_codelet._generated_insights_mo = generated_insights_mo

# Execute the InsightGeneratorCodelet
insight_generator_codelet.proc()

# Retrieve the generated insights
generated_insights = generated_insights_mo.get_info()

print("\nGenerated Insights:")
for insight in generated_insights:
    print(f"- {insight}")

Generated Questions:
- What are the subject's academic achievements?
- What are the subject's travel experiences?
- What is the subject's occupation?

Generated Insights:
- Klaus Mueller is interested in social sciences and its applications.
- Klaus Mueller values knowledge and personal growth.
- Klaus Mueller is likely an extroverted and communicative person.
- Klaus Mueller is probably an academic or student in the field of urban development.
- Klaus Mueller is concerned about the community and its well-being.
